In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50
from torchvision.datasets import CIFAR100
import torch.nn.functional as F

# Load CIFAR100 dataset
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5074, 0.4867, 0.4416), (0.2675, 0.2565, 0.2761))  # CIFAR100 mean and std
])

trainset = CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = CIFAR100(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

# Model: ResNet50 with CBAM
class ResNet50WithCBAM(nn.Module):
    def __init__(self):
        super(ResNet50WithCBAM, self).__init__()
        self.resnet50 = resnet50(pretrained=True)
        self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, 100)  # CIFAR-100 has 100 classes

    def forward(self, x):
        x = self.resnet50(x)
        return x

# Model: ResNet50 without CBAM
class ResNet50WithoutCBAM(nn.Module):
    def __init__(self):
        super(ResNet50WithoutCBAM, self).__init__()
        self.resnet50 = resnet50(pretrained=True)
        self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, 100)  # CIFAR-100 has 100 classes

    def forward(self, x):
        x = self.resnet50(x)
        return x

# Train function (show current and best training accuracy after each epoch)
def train(model, trainloader, criterion, optimizer, device, num_epochs=20):
    best_accuracy = 0.0  # Initialize the best accuracy to track the best model
    for epoch in range(num_epochs):
        model.train()
        correct = 0
        total = 0
        for inputs, targets in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, predicted = outputs.topk(1, 1, True, True)
            correct += (predicted[:, 0] == targets).sum().item()
            total += targets.size(0)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        # Calculate current accuracy for this epoch
        current_accuracy = correct / total * 100

        # Update the best accuracy if current accuracy is higher
        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy

        print(f'Epoch [{epoch+1}/{num_epochs}], Current Accuracy: {current_accuracy:.2f}%, Best Accuracy: {best_accuracy:.2f}%')

# Setup device, models, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_with_cbam = ResNet50WithCBAM().to(device)
model_without_cbam = ResNet50WithoutCBAM().to(device)

criterion = nn.CrossEntropyLoss()
optimizer_with_cbam = optim.SGD(model_with_cbam.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
optimizer_without_cbam = optim.SGD(model_without_cbam.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Train the models
print("Training ResNet50 with CBAM...")
train(model_with_cbam, trainloader, criterion, optimizer_with_cbam, device, num_epochs=20)

print("Training ResNet50 without CBAM...")
train(model_without_cbam, trainloader, criterion, optimizer_without_cbam, device, num_epochs=20)


Files already downloaded and verified
Files already downloaded and verified
Training ResNet50 with CBAM...
Epoch [1/20], Current Accuracy: 7.52%, Best Accuracy: 7.52%
Epoch [2/20], Current Accuracy: 15.82%, Best Accuracy: 15.82%
Epoch [3/20], Current Accuracy: 20.54%, Best Accuracy: 20.54%
Epoch [4/20], Current Accuracy: 23.08%, Best Accuracy: 23.08%
Epoch [5/20], Current Accuracy: 27.05%, Best Accuracy: 27.05%
Epoch [6/20], Current Accuracy: 30.56%, Best Accuracy: 30.56%
Epoch [7/20], Current Accuracy: 32.70%, Best Accuracy: 32.70%
Epoch [8/20], Current Accuracy: 34.31%, Best Accuracy: 34.31%
Epoch [9/20], Current Accuracy: 35.96%, Best Accuracy: 35.96%
Epoch [10/20], Current Accuracy: 37.05%, Best Accuracy: 37.05%
Epoch [11/20], Current Accuracy: 38.28%, Best Accuracy: 38.28%
Epoch [12/20], Current Accuracy: 38.78%, Best Accuracy: 38.78%
Epoch [13/20], Current Accuracy: 39.53%, Best Accuracy: 39.53%
Epoch [14/20], Current Accuracy: 40.27%, Best Accuracy: 40.27%
Epoch [15/20], Current

In [6]:
import torch

# Evaluation function for top-1 and top-5 accuracy
def evaluate(model, dataloader, device):
    model.eval()
    top1 = 0
    top5 = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)

            # Top-1 accuracy
            _, predicted = outputs.topk(1, 1, True, True)
            top1 += (predicted[:, 0] == targets).sum().item()

            # Top-5 accuracy
            _, predicted_top5 = outputs.topk(5, 1, True, True)
            top5 += (predicted_top5 == targets.unsqueeze(1)).sum().item()

    top1_accuracy = top1 / len(dataloader.dataset) * 100
    top5_accuracy = top5 / len(dataloader.dataset) * 100
    return top1_accuracy, top5_accuracy


# Test the models
def test(models, testloader, device):
    for model_name, model in models.items():
        print(f"\nEvaluating {model_name}:")
        top1, top5 = evaluate(model, testloader, device)
        print(f"Top-1 Accuracy: {top1:.2f}%")
        print(f"Top-5 Accuracy: {top5:.2f}%")


# Evaluate both models
models = {
    "ResNet50 with CBAM": model_with_cbam,
    "ResNet50 without CBAM": model_without_cbam
}

# Test the models on the test set
test(models, testloader, device)



Evaluating ResNet50 with CBAM:
Top-1 Accuracy: 37.02%
Top-5 Accuracy: 68.85%

Evaluating ResNet50 without CBAM:
Top-1 Accuracy: 35.33%
Top-5 Accuracy: 67.03%
